In [4]:
# ==============================================================================
# NOTEBOOK 01: ETL DOS DADOS DO IBGE
# Objetivo: Ler o arquivo bruto e salvar uma versão limpa e processada.
# ==============================================================================

import pandas as pd
import io

def processar_dados_ibge(caminho_do_arquivo: str) -> pd.DataFrame:
    """
    Encapsula todo o pipeline de ETL para ler, limpar, transformar e unificar
    os dados de produtividade agrícola do arquivo de relatório do IBGE.
    """
    
    VALORES_FALTANTES = ['-', '...', 'X', '..']

    # --- Funções Auxiliares (Helpers) ---
    def extrair_tabela_por_variavel(linhas_do_arquivo: list, nome_da_variavel: str) -> pd.DataFrame:
        """Encontra e extrai a tabela de uma variável específica do arquivo."""
        bloco_de_dados, cabecalho = [], None
        try:
            indice_inicio_bloco = [i for i, linha in enumerate(linhas_do_arquivo) if nome_da_variavel in linha][0]
        except IndexError: raise ValueError(f"Bloco não encontrado: {nome_da_variavel}")

        for i in range(indice_inicio_bloco, len(linhas_do_arquivo)):
            linha_atual = linhas_do_arquivo[i]
            if linha_atual.strip().startswith('"Cód.","Município"') and '"2017"' in linha_atual:
                cabecalho = linha_atual
                inicio_dos_dados = -1
                for j in range(i + 1, len(linhas_do_arquivo)):
                    if linhas_do_arquivo[j].split(',')[0].strip('"').isdigit():
                        inicio_dos_dados = j
                        break
                break
        
        if not cabecalho or inicio_dos_dados == -1: raise ValueError(f"Cabeçalho ou dados não encontrados: {nome_da_variavel}")

        for linha_de_dados in linhas_do_arquivo[inicio_dos_dados:]:
            if "Fonte: IBGE" in linha_de_dados: break
            bloco_de_dados.append(linha_de_dados)

        csv_limpo_string = cabecalho + "".join(bloco_de_dados)
        return pd.read_csv(io.StringIO(csv_limpo_string), na_values=VALORES_FALTANTES)

    def transformar_dataframe(df_bruto, nome_da_coluna_valor: str) -> pd.DataFrame:
        """Renomeia, derrete (melts) e limpa um DataFrame de variável única."""
        df_bruto.rename(columns={'Cód.': 'cod_municipio', 'Município': 'municipio_uf'}, inplace=True)
        df_long = pd.melt(df_bruto, 
                          id_vars=['cod_municipio', 'municipio_uf'], 
                          var_name='ano', 
                          value_name=nome_da_coluna_valor)
        
        df_clean = df_long.copy()
        
        df_clean['ano'] = pd.to_numeric(df_clean['ano'])
        df_clean[nome_da_coluna_valor] = pd.to_numeric(df_clean[nome_da_coluna_valor])
        
        df_clean[['municipio_nome', 'uf']] = df_clean['municipio_uf'].str.extract(r'^(.*) \((.*)\)$')
        colunas_finais = ['cod_municipio', 'municipio_nome', 'uf', 'ano', nome_da_coluna_valor]
        df_final = df_clean[colunas_finais]
        return df_final

    # --- Execução do Pipeline ---
    
    with open(caminho_do_arquivo, 'r', encoding='utf-8') as f:
        todas_as_linhas = f.readlines()
        
    df_area_bruta = extrair_tabela_por_variavel(todas_as_linhas, "Área plantada ou destinada à colheita")
    df_prod_bruta = extrair_tabela_por_variavel(todas_as_linhas, "Quantidade produzida")
    df_rend_bruta = extrair_tabela_por_variavel(todas_as_linhas, "Rendimento médio da produção")
    
    df_area = transformar_dataframe(df_area_bruta, 'area_plantada_ha')
    df_prod = transformar_dataframe(df_prod_bruta, 'quantidade_produzida_ton')
    df_rend = transformar_dataframe(df_rend_bruta, 'rendimento_medio_kg_ha')
    
    df_merged = pd.merge(df_area, df_prod, on=['cod_municipio', 'ano', 'municipio_nome', 'uf'], how='outer')
    df_final = pd.merge(df_merged, df_rend, on=['cod_municipio', 'ano', 'municipio_nome', 'uf'], how='outer')
    
    df_final.dropna(subset=['area_plantada_ha', 'quantidade_produzida_ton', 'rendimento_medio_kg_ha'], how='all', inplace=True)
    df_final.sort_values(by=['cod_municipio', 'ano'], inplace=True)
    df_final.reset_index(drop=True, inplace=True)
    
    # LINHA CRUCIAL QUE ESTAVA FALTANDO
    return df_final

# --- Ponto de Entrada Principal ---

# 1. Definir os caminhos
caminho_bruto = '../data/raw/rawAgroMilho.csv'
caminho_processado = '../data/processed/dados_consolidados_produtividade_co_2017-2023.csv'

# 2. Executar o pipeline de ETL
df_processado = processar_dados_ibge(caminho_bruto)

# 3. Salvar o arquivo limpo para uso futuro
df_processado.to_csv(caminho_processado, index=False)

print(f"ETL Concluído. Arquivo limpo salvo em: {caminho_processado}")
df_processado.info()

df_final = pd.read_csv(caminho_processado)
print(f"Total de registros: {len(df_final)}")

print("Setup concluído. DataFrame 'df_final' carregado com sucesso.")
df_final

ETL Concluído. Arquivo limpo salvo em: ../data/processed/dados_consolidados_produtividade_co_2017-2023.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cod_municipio             3026 non-null   int64  
 1   municipio_nome            3026 non-null   object 
 2   uf                        3026 non-null   object 
 3   ano                       3026 non-null   int64  
 4   area_plantada_ha          3026 non-null   float64
 5   quantidade_produzida_ton  3025 non-null   float64
 6   rendimento_medio_kg_ha    3025 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 165.6+ KB
Total de registros: 3026
Setup concluído. DataFrame 'df_final' carregado com sucesso.


,cod_municipio,municipio_nome,uf,ano,area_plantada_ha,quantidade_produzida_ton,rendimento_medio_kg_ha
0,5000203,Água Clara,MS,2021,342.0,1104.0,4800.0
1,5000203,Água Clara,MS,2022,120.0,598.0,4983.0
2,5000252,Alcinópolis,MS,2017,4620.0,30780.0,6662.0
3,5000252,Alcinópolis,MS,2018,4680.0,28620.0,6115.0
4,5000252,Alcinópolis,MS,2019,6824.0,41784.0,6123.0
...,...,...,...,...,...,...,...
3021,5300108,Brasília,DF,2019,64000.0,499800.0,7809.0
3022,5300108,Brasília,DF,2020,61200.0,486138.0,7943.0
3023,5300108,Brasília,DF,2021,60000.0,324000.0,5400.0
3024,5300108,Brasília,DF,2022,60000.0,327000.0,5450.0
